In [1]:
#Make the necessary imports
import os
import sys
import numpy as np
import tarfile
import warnings
warnings.filterwarnings("ignore")
from zipfile import ZipFile
!pip install wget
import wget
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=e28a7d72c7dd54544d359e6429847b10095bda6c97a6f6c7062ae9d2897fc7f6
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
%%capture

try:
  from google.colab import files
  !wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
  !unzip DATAPATH/glove.6b.zip -d DATAPATH/glove.6B

  !wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
  !tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH

  BASE_DIR='DATAPATH'

except ModuleNotFoundError:

  if not os.path.exists('Data/glove.6B'):
    os.mkdir('Data/glove.6B')

    url='http://nlp.stanford.edu/data/glove.6B.zip'
    wget.download(url, 'Data')

    temp='Data/glove.6B.zip'
    file=ZipFile(temp)
    file.extractall('Data/glove.6B')
    file.close()

    if not os.path.exists('Data/aclImdb'):

        url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
        wget.download(url,'Data')

        temp='Data/aclImdb_v1.tar.gz'
        tar = tarfile.open(temp, "r:gz")
        tar.extractall('Data')
        tar.close()

    BASE_DIR = 'Data'

In [15]:
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [18]:
!kaggle datasets download -d danielwillgeorge/glove6b100dtxt

Dataset URL: https://www.kaggle.com/datasets/danielwillgeorge/glove6b100dtxt
License(s): unknown
 99% 130M/131M [00:01<00:00, 103MB/s]
100% 131M/131M [00:01<00:00, 107MB/s]


In [19]:
!unzip glove6b100dtxt.zip -d DATAPATH/glove.6B/

Archive:  glove6b100dtxt.zip
  inflating: DATAPATH/glove.6B/glove.6B.100d.txt  


In [3]:
GLOVE_DIR=os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR=os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [4]:
MAX_SEQUENCE_LENGTH=1000
MAX_NUM_WORDS=20000
EMBEDDING_DIM=100
VALIDATION_SPLIT=0.2


In [5]:
#@ Loading and preprocessing:
def get_data(data_dir):
  texts=[]
  labels_index={'pos':1, 'neg':0}
  labels=[]
  for name in sorted(os.listdir(data_dir)):
    path=os.path.join(data_dir, name)
    if os.path.isdir(path):
      if name=='pos' or name=='neg':
        label_id=labels_index[name]
        for fname in sorted(os.listdir(path)):
          fpath=os.path.join(path, fname)
          text=open(fpath, encoding='utf8').read()
          texts.append(text)
          labels.append(label_id)

  return texts, labels

train_texts, train_labels=get_data(TRAIN_DATA_DIR)
test_texts, test_labels=get_data(TEST_DATA_DIR)
labels_index={'pos':1, 'neg':0}

#Just to see how the data looks like.
print(train_texts[0])
print(train_labels[0])
print(test_texts[24999])
print(test_labels[24999])

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.
0
I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary conviction, to save lives. After losing his team he takes a teaching posi

In [6]:
# vectorize these text samples into to 2D integer tensors using Keras Tokenizer
# tokenizer is fit on training data:

tokenizer=Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences=tokenizer.texts_to_sequences(test_texts)
word_index=tokenizer.word_index
print('Found %s unique tokens.' %len(word_index))

Found 88582 unique tokens.


In [8]:
#@ converting to this sequence to fed in nn:
trainvalid_data=pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data=pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels=to_categorical(np.asarray(train_labels))
test_labels=to_categorical(np.asarray(test_labels))


In [20]:
#@ Using Pretrained embeddings:
embeddings_index={} #for mapping corresponding vector representation
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8') as f:
  for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:], dtype='float32')
    embeddings_index[word]=coefs

num_words=min(MAX_NUM_WORDS, len(word_index)) +1
embedding_matrix=np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
  if i> MAX_NUM_WORDS:
    continue
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing of embedding matrix is done


In [21]:
# split the training data into a training set and a validation set
indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])
x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]
#This is the data we will use for CNN and RNN training

### LSTM Model using pre-trained Embedding Layer

In [22]:
rnnmodel=Sequential()
rnnmodel.add(embedding_layer)
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(2, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
print('training RNN')

rnnmodel.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_val, y_val))
score, acc=rnnmodel.evaluate(test_data, test_labels, batch_size=32)
print('Test accuracy with RNN:', acc)

training RNN
625/625 ━━━━━━━━━━━━━━━━━━━━ 702s 1s/step - accuracy: 0.6114 - loss: 0.6512 - val_accuracy: 0.7542 - val_loss: 0.5568
782/782 ━━━━━━━━━━━━━━━━━━━━ 375s 479ms/step - accuracy: 0.7084 - loss: 0.5714
Test accuracy with RNN: 0.7523199915885925
